In [ ]:
# !pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git
# !pip install pandas
# !pip install protobuf
# !pip install -U scikit-learn

In [2]:
from parrot import Parrot
from difflib import SequenceMatcher
import torch
import difflib
import pandas
import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [4]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
phrases = ["Thank you for teaching this good.",
           "Explain both Thai and English. Covering the whole lesson. Thank you very much.",
           "Would like to have more HW work than the exam"]

In [39]:
augment_percents = []
ranker = "levenshtein" # levenshtein, euclidean, diff

for phrase in phrases:
  print("-"*100)
  para_phrases = parrot.augment(input_phrase=phrase,
                                use_gpu=False,
                                diversity_ranker=ranker, 
                                max_return_phrases = 10, 
                                max_length=32, 
                                )

  # for para_phrase in para_phrases:
  print("Original:  " + re.sub('\s+', ' ', str(phrase).strip()))
  print("Augmented: " + re.sub('\s+', ' ', str(para_phrases[0][0]).strip()))
  a = re.sub('\s+', ' ', str(phrase).strip())
  b = re.sub('\s+', ' ', str(para_phrases[0][0]).strip())
  
  augment_percent = round(1 - SequenceMatcher(a=a, b=b).ratio(), 2) * 100
  print("Percent of changing: " + str(format(augment_percent, ".2f")) + "%")
  augment_percents.append(augment_percent)

print("-"*100)
print("Average: " + str(round(sum(augment_percents) / len(augment_percents), 2)))

----------------------------------------------------------------------------------------------------
Original:  Thank you for teaching this good.
Augmented: thank you for the good teaching
Percent of changing: 34.00%
----------------------------------------------------------------------------------------------------
Original:  Explain both Thai and English. Covering the whole lesson. Thank you very much.
Augmented: Explain both Thai and English. Covering the whole lesson. Thank you very much.
Percent of changing: 0.00%
----------------------------------------------------------------------------------------------------
Original:  Would like to have more HW work than the exam
Augmented: Would like to have more HW work than the exam
Percent of changing: 0.00%
----------------------------------------------------------------------------------------------------
Average: 11.33
